<a href="https://colab.research.google.com/github/VellummyilumVinoth/Aspect_based_sentimental_analysis/blob/main/Apect_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [2]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [3]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [4]:
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.2 MB/s eta 0:00:00


In [5]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader 
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification
from transformers import logging
from sklearn.model_selection import train_test_split
import numpy as np

logging.set_verbosity_warning()


In [7]:
from torch import cuda
device = torch.device('cuda' if cuda.is_available() else 'cpu')

print(f"Current device: {device}")

Current device: cpu


In [8]:
# Step 1: Prepare your dataset
df = pd.read_csv('/content/drive/MyDrive/zidan/aspect_extracted_file.csv')

train_data, test_data = train_test_split(df, test_size=0.2)

In [9]:
print(f"Total Training Records : {len(train_data)}")
train_data.head(10)

Total Training Records : 8845


,product_name,review,preprocess_text,aspects,price,quality,delivery cost,packaging
3102,"Scott ComfortPlus Toilet Paper, 36 Mega Rolls,...",Love this toilet paper!,"['love', 'toilet', 'paper']",['price'],1,0,0,0
7094,"Scott 75143 Scott Shop Towels, Blue (3 Rolls, ...","GREAT SHOP TOWELS, VERY ABSORBANT AND DURABLE","['great', 'shop', 'towels', 'absorbant', 'dura...",['price'],1,0,0,0
207,"Salate Cordless Electric Spin Scrubber, Electr...",i used this to clean my bathroom and this work...,"['used', 'clean', 'bathroom', 'works', 'wonder...",['price'],1,0,0,0
1373,Scrub Daddy Scrub Mommy - Scratch-Free Multipu...,I love the dual side aspect of this sponge. It...,"['love', 'dual', 'side', 'aspect', 'sponge', '...",['price'],1,0,0,0
6024,Broom and Dustpan Set for Home - Premium Long ...,"Good height, holds up steady and well . It’s c...","['good', 'height', 'holds', 'steady', 'well', ...","['price', 'quality']",1,1,0,0
8143,Reynolds Wrappers Pre-Cut Aluminum Foil Sheets...,"Great product, but bought in April 2022 for $2...","['great', 'product', 'bought', 'april', '2022'...",['price'],1,0,0,0
2783,"Lysol Click Gel Automatic Toilet Bowl Cleaner,...",I used to buy the one with dispenser but these...,"['used', 'buy', 'one', 'dispenser', 'single', ...",['price'],1,0,0,0
5202,Brandon-super Disposable Food Prep Gloves - 50...,Like it,['like'],['price'],1,0,0,0
4987,PAMI Medium-Weight Disposable Plastic Forks [4...,I have 4 young kids and these are perfect for ...,"['4', 'young', 'kids', 'perfect', 'use', 'sinc...",['price'],1,0,0,0
1195,"Sparkle® Tear-A-Square® Paper Towels, 2 Double...","Not the thickest paper towel, but for the pric...","['thickest', 'paper', 'towel', 'price', 'econo...",['price'],1,0,0,0


In [10]:
columns_to_drop = ['product_name', 'aspects', 'preprocess_text']
train_data.drop(columns_to_drop, axis=1, inplace=True)
train_data['labels'] = train_data.iloc[:, 1:].values.tolist()
train_data.drop(train_data.columns.values[1:-1].tolist(), inplace=True, axis=1)
train_data.head()

,review,labels
3102,Love this toilet paper!,"[1, 0, 0, 0]"
7094,"GREAT SHOP TOWELS, VERY ABSORBANT AND DURABLE","[1, 0, 0, 0]"
207,i used this to clean my bathroom and this work...,"[1, 0, 0, 0]"
1373,I love the dual side aspect of this sponge. It...,"[1, 0, 0, 0]"
6024,"Good height, holds up steady and well . It’s c...","[1, 1, 0, 0]"


In [11]:
train_data["review"] = train_data["review"].str.lower()
train_data["review"] = train_data["review"].str.replace("\xa0", " ", regex=False).str.split().str.join(" ")

In [12]:
train_data.head(10)

,review,labels
3102,love this toilet paper!,"[1, 0, 0, 0]"
7094,"great shop towels, very absorbant and durable","[1, 0, 0, 0]"
207,i used this to clean my bathroom and this work...,"[1, 0, 0, 0]"
1373,i love the dual side aspect of this sponge. it...,"[1, 0, 0, 0]"
6024,"good height, holds up steady and well . it’s c...","[1, 1, 0, 0]"
8143,"great product, but bought in april 2022 for $2...","[1, 0, 0, 0]"
2783,i used to buy the one with dispenser but these...,"[1, 0, 0, 0]"
5202,like it,"[1, 0, 0, 0]"
4987,i have 4 young kids and these are perfect for ...,"[1, 0, 0, 0]"
1195,"not the thickest paper towel, but for the pric...","[1, 0, 0, 0]"


In [13]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
EPOCHS = 1
LEARNING_RATE = 2e-05
NUM_WORKERS = 2

In [14]:
from torch.utils.data import Dataset, DataLoader 

class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len: int, eval_mode: bool = False):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.text = dataframe.review
        self.eval_mode = eval_mode 
        if self.eval_mode is False:
            self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text.iloc[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        output = {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }
                
        if self.eval_mode is False:
            output['targets'] = torch.tensor(self.targets.iloc[index], dtype=torch.float)
                
        return output


In [15]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)


In [16]:
training_set[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'ids': tensor([  101,  2293,  2023, 11848,  3259,   999,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,  

In [17]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': NUM_WORKERS
                }
training_loader = DataLoader(training_set, **train_params)

In [24]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class DistilBERTClass(torch.nn.Module):

    def __init__(self):
        super(DistilBERTClass, self).__init__()
        self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 4)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output


In [25]:
model = DistilBERTClass()
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [26]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [27]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [28]:
def train(epoch):
    
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%100==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        loss.backward()
        optimizer.step()

In [29]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]

Epoch: 0, Loss:  0.6897130012512207


1it [00:43, 43.78s/it]


KeyboardInterrupt: ignored

In [30]:
test_data.head()

,product_name,review,preprocess_text,aspects,price,quality,delivery cost,packaging
5321,"[3 Pack] Mop Replace Heads, Spin Mop Refill He...",Like that the size is much bigger than the ori...,"['like', 'size', 'much', 'bigger', 'original',...",['price'],1,0,0,0
5956,"AIDEA Heavy Duty Scrub Sponge-24Count, Cleanin...",Needed to replenish/sturdy,"['needed', 'replenish/sturdy']",['price'],1,0,0,0
10086,"Method Fabric Softener, Beach Sage, 45 loads, ...",Laundry,['laundry'],['price'],1,0,0,0
5892,"Shower Squeegee, 10-Inch Matte Black Squeegee,...","I like the 18"" --- 3 or 4 swipes NOT 10 or mor...","['like', '18', '3', '4', 'swipes', '10', 'like...",['price'],1,0,0,0
10550,"Energizer AA Batteries, Max Double A Battery A...","If you don’t “mark” one time only,they will ju...","['dont', '“', 'mark', '”', 'one', 'time', 'onl...",['price'],1,0,0,0


In [33]:
columns_to_keep = ['review']
test_data = test_data[columns_to_keep]

In [34]:
test_data.head()

,review
5321,Like that the size is much bigger than the ori...
5956,Needed to replenish/sturdy
10086,Laundry
5892,"I like the 18"" --- 3 or 4 swipes NOT 10 or mor..."
10550,"If you don’t “mark” one time only,they will ju..."


In [ ]:
test_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN, eval_mode = True)
testing_params = {'batch_size': TRAIN_BATCH_SIZE,
               'shuffle': True,
               'num_workers': NUM_WORKERS
                }
test_loader = DataLoader(test_set, **testing_params)

In [ ]:
all_test_pred = []

def test(epoch):
    model.eval()
    
    with torch.inference_mode():
    
        for _, data in tqdm(enumerate(test_loader, 0)):

            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            outputs = model(ids, mask, token_type_ids)
            probas = torch.sigmoid(outputs)

            rounded_probas = torch.round(probas)  # Round probabilities to 0 or 1

            all_test_pred.append(rounded_probas)

    return probas

In [ ]:
probas = test(model)

In [ ]:
all_test_pred = torch.cat(all_test_pred)

In [ ]:
submit_df = test_data.copy()

In [ ]:
label_columns = ["price", "packaging", "quality", "delivery cost"]

In [ ]:
for i,name in enumerate(label_columns):

    submit_df[name] = all_test_pred[:, i].cpu()

In [ ]:
submit_df.head(10)

In [ ]:
# Predict the aspects for a sample review
sample_review = "The product packaging was excellent."
sample_encoding = tokenizer.encode_plus(sample_review, truncation=True, padding=True, return_tensors='pt')
sample_input_ids = sample_encoding['input_ids'].to(device)
sample_attention_mask = sample_encoding['attention_mask'].to(device)

with torch.no_grad():
    sample_outputs = model(sample_input_ids, attention_mask=sample_attention_mask)
    _, sample_predicted = torch.max(sample_outputs.logits, dim=1)
    predicted_aspect = aspects[sample_predicted.item()]
    print(f"Predicted aspect for the sample review: {predicted_aspect}")